[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/114_nasa_fire.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/114_nasa_fire.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**Visualizing NASA Fire Perimeter Data**

This notebook demonstrates how to access and visualize NASA fire perimeter data using leafmap. The data comes from the [Fire Event Data Suite (FEDs)](https://www.earthdata.nasa.gov/data/tools/firms) algorithm, which processes VIIRS sensor data from Suomi NPP and NOAA-20 satellites to track active fires and fire perimeters.

The data is accessed via the [OpenVEDA OGC API Features](https://openveda.cloud/api/features) endpoint.

Uncomment the following line to install [leafmap](https://leafmap.org) if needed.

In [ ]:
# %pip install -U leafmap geopandas osmnx

In [ ]:
import leafmap

## List Available Fire Collections

First, let's see what fire data collections are available:

In [ ]:
# List available fire collections
collections = leafmap.get_fire_collections(verbose=True)

The main fire collections are:

| Collection | Description |
|------------|-------------|
| `snapshot_perimeter_nrt` | 20-day recent fire perimeters from VIIRS |
| `lf_perimeter_nrt` | Current year large fires (>5 km²) |
| `lf_perimeter_archive` | 2018-2021 Western US archived large fires |
| `lf_fireline_nrt` | Active fire lines for large fires |
| `lf_newfirepix_nrt` | New fire pixels detected |

**Note:** The NRT (Near Real-Time) collections contain rolling data from the past 20 days. For historical analysis, use the archive collection.

## Retrieve Fire Data by Bounding Box

Let's retrieve recent fire data for California:

In [ ]:
# Define a bounding box for California [west, south, east, north]
bbox = [-124.48, 32.53, -114.13, 42.01]

# Get recent fire perimeters
gdf = leafmap.fire_gdf_from_bbox(bbox, collection="snapshot_perimeter_nrt")
print(f"Found {len(gdf)} fire perimeters")
gdf.head()

## Visualize Fire Perimeters on a Map

Let's display the fire perimeters on an interactive map:

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
if not gdf.empty:
    m.add_gdf(
        gdf,
        layer_name="Fire Perimeters",
        style={
            "color": "#FF4500",
            "weight": 2,
            "fillColor": "#FF6347",
            "fillOpacity": 0.5,
        },
    )
m

## Using the add_fire_data Method

The `add_fire_data` method provides a convenient way to fetch and display fire data directly on the map:

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_fire_data(
    bbox=[-124.48, 32.53, -114.13, 42.01],
    collection="snapshot_perimeter_nrt",
    layer_name="California Fires",
)
m

## Retrieve Fire Data by Place Name

You can also retrieve fire data by place name using geocoding:

In [ ]:
# Get fire data for Oregon
gdf = leafmap.fire_gdf_from_place("Oregon, USA", collection="snapshot_perimeter_nrt")
print(f"Found {len(gdf)} fire perimeters in Oregon")
gdf.head()

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_fire_data(
    place="Oregon, USA",
    collection="snapshot_perimeter_nrt",
    layer_name="Oregon Fires",
)
m

## Filter by Fire Properties

You can filter fires based on their properties such as fire area, duration, and mean Fire Radiative Power (FRP):

In [ ]:
# Get only fires with area >= 1 km²
gdf = leafmap.search_fires(
    place="California",
    collection="snapshot_perimeter_nrt",
    farea_min=1,  # Minimum fire area in km²
)
print(f"Found {len(gdf)} fire perimeters (>= 1 km²)")
gdf.head()

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
if not gdf.empty:
    m.add_gdf(
        gdf,
        layer_name="Fires >= 1 km²",
        style={
            "color": "#FF0000",
            "weight": 3,
            "fillColor": "#FF4500",
            "fillOpacity": 0.6,
        },
    )
m

## Large Fires from the Current Year

The `lf_perimeter_nrt` collection contains large fires (>5 km²) from the current year:

In [ ]:
# Get current year large fires
gdf = leafmap.search_fires(
    place="California",
    collection="lf_perimeter_nrt",
)
print(f"Found {len(gdf)} large fire perimeters")
if not gdf.empty:
    print(f"\nDate range: {gdf['t'].min()} to {gdf['t'].max()}")
    gdf.head()

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
if not gdf.empty:
    m.add_gdf(
        gdf,
        layer_name="Large Fires",
        style={
            "color": "#FF4500",
            "weight": 2,
            "fillColor": "#FF6347",
            "fillOpacity": 0.5,
        },
    )
m

## Color-Coded Fire Perimeters

Use the `add_fire_perimeters` method to display fires with color scaling based on fire area or other properties:

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_fire_perimeters(
    place="California",
    collection="snapshot_perimeter_nrt",
    color_column="farea",
    cmap="YlOrRd",
    legend_title="Fire Area (km²)",
)
m

## Archived Fire Data (2018-2021)

Access historical fire data from 2018-2021 Western US. This collection allows filtering by date range:

In [ ]:
# Get archived fire data for summer 2020
gdf = leafmap.search_fires(
    bbox=[-124.48, 32.53, -114.13, 42.01],
    collection="lf_perimeter_archive",
    datetime="2020-07-01/2020-10-31",
)
print(f"Found {len(gdf)} archived fire perimeters from summer 2020")
gdf.head()

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
if not gdf.empty:
    m.add_gdf(
        gdf,
        layer_name="2020 Fire Season",
        style={
            "color": "#FF4500",
            "weight": 2,
            "fillColor": "#FF6347",
            "fillOpacity": 0.5,
        },
    )
m

## Fire Attributes

The fire data includes several useful attributes:

| Attribute | Description |
|-----------|-------------|
| `fireid` | Unique fire identifier |
| `farea` | Fire area in km² |
| `fperim` | Fire perimeter in km |
| `meanfrp` | Mean Fire Radiative Power |
| `duration` | Fire duration in days |
| `t` | Timestamp of the observation |
| `n_pixels` | Number of VIIRS pixels |
| `isactive` | Whether the fire is currently active |

In [ ]:
# Examine fire attributes
gdf = leafmap.fire_gdf_from_bbox(
    [-124.48, 32.53, -114.13, 42.01], collection="snapshot_perimeter_nrt"
)
if not gdf.empty:
    print("Available columns:")
    print(gdf.columns.tolist())
    print("\nSample data:")
    print(gdf[["fireid", "farea", "fperim", "duration", "t", "isactive"]].head())

## Export Fire Data

You can export the fire data to various formats for further analysis:

In [ ]:
# Export to GeoJSON
# gdf.to_file("california_fires.geojson", driver="GeoJSON")

# Export to GeoParquet
# gdf.to_parquet("california_fires.parquet")

# Export to Shapefile
# gdf.to_file("california_fires.shp")

## Summary

This notebook demonstrated how to:

1. List available NASA fire data collections
2. Retrieve fire perimeter data by bounding box or place name
3. Filter fires by fire properties (area, duration, etc.)
4. Visualize fire perimeters with custom styling
5. Create color-coded maps based on fire attributes
6. Access archived fire data (2018-2021)
7. Export data for further analysis

**Note:** The NRT collections (`snapshot_perimeter_nrt`, `lf_perimeter_nrt`) contain rolling 20-day data. For historical analysis, use `lf_perimeter_archive` which covers 2018-2021.

For more information about the FEDs algorithm and data, visit:
- [NASA FIRMS](https://firms.modaps.eosdis.nasa.gov/)
- [OpenVEDA](https://openveda.cloud/)